In [ ]:
from helper import *
from postProcessHelper import *

config = Config()
config.setFromEnv()

versionControl = 'V3'
promptVersion='P3'
simpleCourseName = 'ECON'
fullName = f'{simpleCourseName}-{versionControl}-{promptVersion}'
config.customDescMode = False
config.setSaveDetails(fullName)

chartFolder = os.path.join(config.baseOutputFolder, \
                           config.outputFolders['CHART_OUTPUT'], \
                           config.fullName)

excelFolder = os.path.join(config.baseOutputFolder, \
                           config.outputFolders['EXCEL_OUTPUT'], \
                           config.fullName)


In [ ]:
resultsDF = convertPicklesToDF('saves', config)
errorDF = convertPicklesToDF('errors', config)

mergedCriterionData = getCriterionDataDF(resultsDF, fullName, excelFolder)

In [ ]:
getScoreSpread(resultsDF, chartFolder)

## Track Difference in scores per Assignment

In [ ]:
def saveGraderPeerGPTMeanScoreDiff(resultsDF, saveName, excelFolder):
    excludeDF = resultsDF.copy()
    excludeDF['Score Difference'] = excludeDF['peerGPT_score']-excludeDF['score']
    excludeDF['Score Diff. %'] = 100*(excludeDF['peerGPT_score']-excludeDF['score'])/excludeDF['points_possible']

    meanDiffDict = {}
    meanDiffPercentDict = {}
    for group in excludeDF.groupby(['grader_id','assignment_id']):
        if group[0][0] not in meanDiffDict:
            meanDiffDict[group[0][0]] = {}
            meanDiffPercentDict[group[0][0]] = {}
        meanDiffDict[group[0][0]][group[0][1]] = group[1]["Score Difference"].mean()
        meanDiffPercentDict[group[0][0]][group[0][1]] = np.round(group[1]["Score Diff. %"].mean() , 2)
    meanDiffDF = pd.DataFrame(meanDiffDict)
    meanDiffPercentDF = pd.DataFrame(meanDiffPercentDict)
    meanDiffDF.to_excel(os.path.join(excelFolder, saveName+' - Grader - peerGPT Score Difference.xlsx'))
    meanDiffPercentDF.to_excel(os.path.join(excelFolder, saveName+' - Grader - peerGPT Score Diff. %.xlsx'))
    return meanDiffPercentDF

saveGraderPeerGPTMeanScoreDiff(resultsDF, fullName, excelFolder)

## Track difference in score per Criteria

In [ ]:
fullInfoDF, rubricOrderDict = buildFullInfoDF(config, resultsDF, fullName, excelFolder)

In [ ]:
getMeanDiffPercentCharts(fullInfoDF, rubricOrderDict, chartFolder)

In [ ]:
meanConDF, retrievedCIDF = getCIOutlierGraderDetails(fullInfoDF, fullName, excelFolder, confidenceLevel=0.99)

display(meanConDF)

In [ ]:
getHighErrorCriteria(config, fullInfoDF, fullName, excelFolder, scoreThreshold = 10)

## Generate Histogram spread of scores by Assignment

In [ ]:
getFullHistogramSpread(resultsDF, chartFolder)

## Generate Scatterplot spread of scores by Assignment

In [ ]:
getFullScatterplotSpread(resultsDF, chartFolder)

## Generate Histogram spread of scores by Criterion per Assignment

In [ ]:
getHistogramSpreadByAssgn(resultsDF, chartFolder)

## Generate Scatterplot spread of scores by Criterion per Assignment

In [ ]:
getScatterplotSpreadByAssgn(resultsDF, chartFolder)

## Misc. Code Snippets

In [ ]:
getMeanDiffCharts(fullInfoDF, rubricOrderDict, chartFolder)

In [ ]:
from helper import *

config = Config()
config.setFromEnv()
config.customDescMode = False

versionControl = 'V3'
promptVersion='P2'
simpleCourseName = 'ECON'
fullName = f'{simpleCourseName}-{versionControl}-{promptVersion}'
config.setSaveDetails(fullName)

gradeRubricAssignmentDF = getGRAData(config)

for index, row in gradeRubricAssignmentDF.iterrows():
    fullPrompt = processTokenCount(row, config)

    if fullPrompt:
        with open(os.path.join(config.baseOutputFolder, \
                               config.outputFolders['PROMPT_FILES'], \
                               config.fullName, \
                               f'{config.fullName}_exampleFilledPrompt.txt'), 'w') as textFile:
            textFile.write(fullPrompt)
        break

In [ ]:
import shutil
from helper import *

config = Config()

sourceFolder = os.path.join(config.baseOutputFolder, config.outputFolders['PROMPT_FILES'])
destinationFolder = config.promptFolder

if os.path.exists(destinationFolder):
    shutil.rmtree(destinationFolder)
shutil.copytree(sourceFolder, destinationFolder)

In [ ]:
P2DF = pd.read_excel('output\\Excel File Outputs\\ECON-V3-P2\\ECON-V3-P2 - Grader - peerGPT Score Diff. %.xlsx')
P3DF = pd.read_excel('output\\Excel File Outputs\\ECON-V3-P3\\ECON-V3-P3 - Grader - peerGPT Score Diff. %.xlsx')

display(P2DF)
display(P3DF)
display(P2DF-P3DF)